In [1]:
%load_ext autoreload
%autoreload 2
import torch
torch.cuda.is_available()

True

In [2]:
# Test the function generating warp matrix from vector of numbers between 0 and 1

from reshape import WarpMatrix
from torch.autograd import Variable
from utils import to_gpu
import torch


#w=to_gpu(Variable(torch.FloatTensor(5,1).uniform_(), requires_grad = True))
w=Variable(to_gpu(torch.FloatTensor(5,1).uniform_()), requires_grad = True)
wmat = WarpMatrix.apply(w)
loss = wmat.sum(0)[0]
loss.backward()
print(wmat, loss, w.grad)
a = w.grad.data.zero_()

Variable containing:
 0.4212  0.4983  0.0805  0.0000  0.0000
 0.0000  0.0000  0.8173  0.1827  0.0000
 0.0000  0.0000  0.0000  0.5424  0.4576
 0.0000  0.0000  0.0000  0.0000  0.0964
 0.0000  0.0000  0.0000  0.0000  0.0000
[torch.cuda.FloatTensor of size 5x5 (GPU 0)]
 Variable containing:
 0.4212
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 1
 0
 0
 0
 0
[torch.cuda.FloatTensor of size 5x1 (GPU 0)]



In [3]:
# now let's test the whole fitted-weights-compression transform
from reshape import FittedWarp
from torch.autograd import Variable
import torch

true_w=Variable(to_gpu(torch.FloatTensor(5,1).uniform_()), requires_grad = True)

warp = FittedWarp(true_w.shape)
x = Variable(to_gpu(torch.randn(20,true_w.shape[0])))
out = warp.forward(x)
loss = out.sum()
loss.backward()
print(out, loss,warp.w.grad)
# a =w.grad.data.zero_()

Variable containing:
-0.6468 -0.2843 -0.5138 -0.2249  0.2218
-1.2945 -0.2694 -0.8587 -0.6530 -0.7320
-0.4633  0.1160 -1.7733 -0.8602  0.5088
 0.3132  0.9833 -1.1292 -0.4555 -0.6932
 0.8422  0.2082  0.0233  0.0203 -0.4404
 1.2655 -0.2257 -1.8238  0.1501  0.1338
-0.8861 -1.4449 -0.6904 -0.4980 -0.9470
-0.0672 -0.1439  0.4930 -1.7946  0.4330
-0.0413  0.0378 -0.5719 -0.5288  0.7495
-0.5348 -0.1367 -0.3537  0.2314 -0.2199
-0.4839  1.0498 -2.4528 -0.5887  0.7737
 0.2252  1.1510 -0.2731 -0.0269 -0.4391
 0.2271  0.1152 -0.1455  0.1239 -0.2068
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000
[torch.cuda.FloatTensor of size 20x5 (GPU 0)]
 Variable containing:
-16.4211
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 5.1338
 4.5506
 3.9229
 3.2347
-0.16

In [5]:
# from visualize import make_dot
# make_dot(loss)

In [ ]:
# now let's fit those weights
from torch.optim import lr_scheduler
from data_sources import data_gen
from fit import fit
true_warp = to_gpu(FittedWarp(true_w.shape))
true_warp.w.data = true_w.data
valid_gen = data_gen(batch_size = 100, batches = 1, model = true_warp)
train_gen = data_gen(batch_size = 64, batches = 100, model = true_warp)

#model = to_gpu(Net(true_w.shape))
model = warp
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = torch.nn.MSELoss()
epochs = 10
save_path = 'test.mdl'

valid_data = next(valid_gen)
train_data = [x for x in train_gen]

fit(train_data = train_data,
        valid_data = valid_data,
        model = model,
        optimizer = optimizer,
        scheduler = scheduler,
        epochs = epochs,
        criterion = criterion,
        save_path=save_path)
print(warp.w, true_warp.w)

epoch  0
we're improving! 0.36869949102401733
successfully saved model
epoch  1
we're improving! 0.35812002420425415
successfully saved model
epoch  2
epoch  3
epoch  4
we're improving! 4.964302206644788e-06
successfully saved model
epoch  5
we're improving! 1.307108588033401e-11
successfully saved model
epoch  6
we're improving! 3.474907861455989e-12
successfully saved model
epoch  7
epoch  8


In [ ]:
from torch.autograd import gradcheck

# gradchek takes a tuple of tensor as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (Variable(torch.randn(20,20).double(), requires_grad=True), Variable(torch.randn(30,20).double(), requires_grad=True),)
test = gradcheck(WarpMatrix.apply, input, eps=1e-6, atol=1e-4)
print(test)

In [ ]:
import torch
print(torch.__version__)
class MyFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        output = torch.sign(input)
        return output
    @staticmethod
    def backward(ctx, grad_output):
        # saved tensors - tuple of tensors, so we need get first
        input, = ctx.saved_variables
#         grad_output[input.ge(1)] = 0
#         grad_output[input.le(-1)] = 0
        return grad_output


# usage
x = torch.autograd.Variable(torch.randn(10, 20), requires_grad = True)
y = MyFunction.apply(x)
# or
# my_func = MyFunction.apply
# y = my_func(x)
loss = y.sum()
loss.backward()

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F


x = Variable(torch.randn(10, 20), requires_grad=False)
y = Variable(torch.randn(10, 3), requires_grad=False)
# define some weights
w1 = Variable(torch.randn(20, 5), requires_grad=True)
w2 = Variable(torch.randn(5, 3), requires_grad=True)

learning_rate = 0.1
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w1, w2], lr=learning_rate)
for step in range(5):
    pred = F.sigmoid(x @ w1)
    pred = F.sigmoid(pred @ w2)
    loss = loss_fn(pred, y)

    # manually zero all previous gradients
    optimizer.zero_grad()
    # calculate new gradients
    loss.backward()
    # apply new gradients
    optimizer.step()
    print(loss)

In [ ]:
import torch

first_counter = torch.Tensor([0])
second_counter = torch.Tensor([10])
some_value = torch.Tensor(15)

while (first_counter < second_counter)[0]:
    first_counter += 2
    second_counter += 1

In [ ]:
next([1].iter())

In [ ]:
from collections import OrderedDict

import torch.nn as nn


# Example of using Sequential
model = nn.Sequential(
    nn.Conv2d(1, 20, 5),
    nn.ReLU(),
    nn.Conv2d(20, 64, 5),
    nn.ReLU()
)

# Example of using Sequential with OrderedDict
model = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 20, 5)),
    ('relu1', nn.ReLU()),
    ('conv2', nn.Conv2d(20, 64, 5)),
    ('relu2', nn.ReLU())
]))

#output = model(some_input)

In [ ]:
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 12, kernel_size=3, padding=1, stride=1),
            nn.Conv2d(12, 24, kernel_size=3, padding=1, stride=1),
        )
        self.second_extractor = nn.Conv2d(
            24, 36, kernel_size=3, padding=1, stride=1)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.second_extractor(x)
        # note that we may call same layer twice or mode
        x = self.second_extractor(x)
        return x

In [ ]:
import torch

class MyFunction(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        output = torch.sign(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        # saved tensors - tuple of tensors, so we need get first
        input, = ctx.saved_variables
        grad_output[input.ge(1)] = 0
        grad_output[input.le(-1)] = 0
        return grad_output


# usage
x = torch.randn(10, 20)
y = MyFunction.apply(x)
# or
my_func = MyFunction.apply
print(MyFunction)
y = my_func(x)


# and if we want to use inside nn.Module
class MyFunctionModule(torch.nn.Module):
    def forward(self, x):
        return MyFunction.apply(x)


In [ ]:

import torch

### tensor example
x_cpu = torch.randn(10, 20)
w_cpu = torch.randn(20, 10)
# direct transfer to the GPU
x_gpu = x_cpu.cuda()
w_gpu = w_cpu.cuda()
result_gpu = x_gpu @ w_gpu
# get back from GPU to CPU
result_cpu = result_gpu.cpu()

### model example
model = model.cuda()
# train step
inputs = Variable(inputs.cuda())
outputs = model(inputs)
# get back from GPU to CPU
outputs = outputs.cpu()


In [ ]:

import torch

# check is cuda enabled
torch.cuda.is_available()

# set required device
torch.cuda.set_device(0)

# work with some required cuda device
with torch.cuda.device(1):
    # allocates a tensor on GPU 1
    a = torch.cuda.FloatTensor(1)
    assert a.get_device() == 1

    # but you still can manually assign tensor to required device
    d = torch.randn(2).cuda(2)
    assert d.get_device() == 2


In [ ]:

import torch
from torch.autograd import Variable

# new way with `init` module
w = torch.Tensor(3, 5)
torch.nn.init.normal(w)
# work for Variables also
w2 = Variable(w)
torch.nn.init.normal(w2)
# old styled direct access to tensors data attribute
w2.data.normal_()

# example for some module
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# for loop approach with direct access
class MyModel(nn.Module):
    def __init__(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()


In [ ]:

# scheduler example
from torch.optim import lr_scheduler

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

for epoch in range(100):
    scheduler.step()
    train()
    validate()

# Train flag can be updated with boolean
# to disable dropout and batch norm learning
model.train(True)
# execute train step
model.train(False)
# run inference step

# CPU seed
torch.manual_seed(42)
# GPU seed
torch.cuda.manual_seed_all(42)

In [ ]:

from collections import OrderedDict

import torch.nn as nn
save_path = 'test.mdl'
model = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 20, 5)),
    ('relu1', nn.ReLU()),
    ('conv2', nn.Conv2d(20, 64, 5)),
    ('relu2', nn.ReLU())
]))

print(model)

# Sequential (
#   (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
#   (relu1): ReLU ()
#   (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
#   (relu2): ReLU ()
# )

# save/load only the model parameters(prefered solution)
torch.save(model.state_dict(), save_path)
model.load_state_dict(torch.load(save_path))

# save whole model
torch.save(model, save_path)
model = torch.load(save_path)


In [ ]:
def to_gpu(x):
    return x.cuda()

class ImagesDataset(torch.utils.data.Dataset):
    pass

class Net(nn.Module):
    pass

model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = torch.nn.MSELoss()

fit_dataset = ImagesDataset(path_to_fit_images)
fit_data_loader = torch.utils.data.DataLoader(fitdataset, batch_size=10)

valid_dataset = ImagesDataset(path_to_valid_images)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=10)

best_valid_loss = float('inf')
for epoch in range(epochs):
    # training
    lr_scheduler.step()
    for inputs, labels in fit_data_loader:
        inputs = Variable(to_gpu(inputs))
        labels = Variable(to_gpu(labels))

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # validation:
    # TODO: spell this out!
    valid_inputs, valid_labels = get_data()
    
    inputs = Variable(to_gpu(valid_inputs))
    labels = Variable(to_gpu(valid_labels))

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    if loss < best_valid_loss:
        best_valid_loss = loss
        # spell_out:
        save_model(model)


In [ ]:
import torch

### tensor example
x_cpu = torch.randn(10, 20)
w_cpu = torch.randn(20, 10)
# direct transfer to the GPU
x_gpu = x_cpu.cuda()
w_gpu = w_cpu.cuda()
result_gpu = x_gpu @ w_gpu
# get back from GPU to CPU
result_cpu = result_gpu.cpu()
print(result_cpu)

### model example
model = model.cuda()
# train step
inputs = Variable(inputs.cuda())
outputs = model(inputs)
# get back from GPU to CPU

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F


x = Variable(torch.randn(10, 20), requires_grad=False)
y = Variable(torch.randn(10, 3), requires_grad=False)
# define some weights
w1 = Variable(torch.randn(20, 5), requires_grad=True)
w2 = Variable(torch.randn(5, 3), requires_grad=True)

learning_rate = 0.1
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w1, w2], lr=learning_rate)
for step in range(5):
    pred = F.sigmoid(x @ w1)
    pred = F.sigmoid(pred @ w2)
    loss = loss_fn(pred, y)

    # manually zero all previous gradients
    optimizer.zero_grad()
    # calculate new gradients
    loss.backward()
    # apply new gradients
    optimizer.step()

In [ ]:
import torch
from torch.autograd import Variable
x = Variable(torch.randn(3, 4), requires_grad=False)
y = Variable(torch.randn(3, 2), requires_grad=False)
# define some weights
w1 = Variable(torch.randn(4, 2), requires_grad=True)
w2 = Variable(torch.FloatTensor(w1.data.numpy()), requires_grad=True)
for i in range(5):
    loss1 = torch.mean((y - x @ w1) ** 2)
    loss2 = torch.mean((y - x @ w2) ** 2)
# calculate the gradients
    loss1.backward()
    loss2.backward()
    print("w1 grad(zeroed)", w1.grad)
    print("w2 grad(not zeroed)", w2.grad)
    w1.grad.data.zero_()
    print('-'*10)